In [ ]:
import sys
new_path = '/home/eltons-pc/Code/federated-sc-depth/src'
sys.path.append(new_path)

In [ ]:
CENTRALIZED_TRAINING_DIRPATH = '/home/eltons-pc/Logs/centralized_sc_depth/kitti/'
CENTRALIZED_TRAINING_ID = '02_05_2023' # 100 epochs with 1000 train batches and all val batches

FEDERATED_TRAINING_ID_FEDSCDEPTH = '' # DEFINE BASED ON NEW EXECUTIONS
FEDERATED_TRAINING_ID_BOFEDSCDEPTH = '03_09_2023_13:07:11' # DISTRIBUTION = IID; TOTAL PARTICIPANTS = 12 ; PARTICIPANTS PER ROUND = 4 ; LOCAL EPOCHS = 3; SEARCH RANGE = 6; MAX LOCAL BATCHES TRAIN = 1000; MAX LOCAL BATCHES VAL = -1; MAX_ROUNDS = 36 ;  

FEDERATED_TRAINING_DIRPATH = '/home/eltons-pc/Logs/federated-sc-depth/'
TRAINING_IDS = [
    FEDERATED_TRAINING_ID_BOFEDSCDEPTH,
    FEDERATED_TRAINING_ID_FEDSCDEPTH,
    CENTRALIZED_TRAINING_ID
]
TRAINING_LABELS = [
    "<b>BO-Enhanced FT (BOFedSCDepth)</b>",
    "<b>Standard FT (FedSCDepth)</b>",
    "<b>CT</b>"
]
TRAINING_CENTRALIZED = [
    False,
    False,
    True
]
ROUND_CAP = 12
NUM_CLIENTS = 12
#MODEL_SIZE_MB = 111.417 # estimated by pytorch
MODEL_SIZE_MB = 106.32868194580078 # estimated in code

In [ ]:
import os
from util import get_dir_size
import pandas as pd
TRAIN_DATASET_DIR = '/home/eltons-pc/Datasets/kitti/training'
FORMATS = ['jpg', 'png']
total_files, total_size_gb, filtered_files, filtered_size_gb = get_dir_size(TRAIN_DATASET_DIR, FORMATS)
df_columns = ['Total Files', 'Total Size (GB)', f'{FORMATS} Files', f'{FORMATS} Size (GB)']
df_rows = [total_files, total_size_gb, filtered_files, filtered_size_gb]
df = pd.DataFrame({'': df_columns, 'Value': df_rows})
df

In [ ]:
show_legend = True
fig_config = dict(x_tick_size=25, y_tick_size=25, legend_size=25, trace_size = 6, show_legend = show_legend, marker_size=18)

In [ ]:
from util import get_federated_training_charts, get_centralized_training_charts, standardize_fig
import plotly.graph_objects as go

test_loss_figures = []
communication_cost_upper_bound_figures = []
communication_cost_lower_bound_figures = []
test_loss_by_communication_upper_bound_cost_figures = []
test_loss_by_communication_lower_bound_cost_figures = []
for i in range(len(TRAINING_IDS)):
    if TRAINING_CENTRALIZED[i]:
        test_loss_by_training_step_fig, communication_cost_by_training_step, test_loss_by_communication_cost_fig = get_centralized_training_charts(
            CENTRALIZED_TRAINING_DIRPATH, TRAINING_IDS[i], TRAINING_LABELS[i], filtered_size_gb, 1, MODEL_SIZE_MB, NUM_CLIENTS, show_legend
        )
        communication_cost_upper_bound_figures.append(communication_cost_by_training_step)
        test_loss_by_communication_upper_bound_cost_figures.append(test_loss_by_communication_cost_fig)
        communication_cost_lower_bound_figures.append(communication_cost_by_training_step)
        test_loss_by_communication_lower_bound_cost_figures.append(test_loss_by_communication_cost_fig)
    else:
        _, _, test_loss_by_training_step_fig, communication_cost_upper_bound_by_training_step, test_loss_by_communication_cost_upper_bound_fig, _ = get_federated_training_charts(
            FEDERATED_TRAINING_DIRPATH, ROUND_CAP, TRAINING_IDS[i], TRAINING_LABELS[i], TRAINING_CENTRALIZED[i], filtered_size_gb, 1, None, show_legend
        )
        _, _, _, communication_cost_lower_bound_by_training_step, test_loss_by_communication_cost_lower_bound_fig, _ = get_federated_training_charts(
            FEDERATED_TRAINING_DIRPATH, ROUND_CAP, TRAINING_IDS[i], TRAINING_LABELS[i], TRAINING_CENTRALIZED[i], filtered_size_gb, 1, None, show_legend, True, False
        )
        communication_cost_upper_bound_figures.append(communication_cost_upper_bound_by_training_step)
        test_loss_by_communication_upper_bound_cost_figures.append(test_loss_by_communication_cost_upper_bound_fig)
        communication_cost_lower_bound_figures.append(communication_cost_lower_bound_by_training_step)
        test_loss_by_communication_lower_bound_cost_figures.append(test_loss_by_communication_cost_lower_bound_fig)
    test_loss_figures.append(test_loss_by_training_step_fig)
    
combined_test_loss_figures = go.Figure()
combined_communication_cost_upper_bound_figures = go.Figure()
combined_communication_cost_lower_bound_figures = go.Figure()
combined_test_loss_by_communication_cost_upper_bound_figures = go.Figure()
combined_test_loss_by_communication_cost_lower_bound_figures = go.Figure()

import random
MARKER_SYMBOLS = ['square', 'circle','cross']
COLORS = ['navy', 'purple', 'green']

i = 0
for fig in test_loss_figures:
    for trace in fig.data:
        trace.update(line={'color': COLORS[i]}, marker=dict(symbol=MARKER_SYMBOLS[i]))
        i += 1
        combined_test_loss_figures.add_trace(trace)
combined_test_loss_figures.update_layout(xaxis_title="<b>Training Step (k)</b>", yaxis_title="<b>Validation Loss (AbsRel)</b>")
combined_test_loss_figures = standardize_fig(combined_test_loss_figures, **fig_config)
combined_test_loss_figures.update_layout(legend=dict(orientation="v", yanchor="bottom", x=0.8, y=0.7, xanchor="left", bgcolor='rgba(0,0,0,0)')) 
combined_test_loss_figures.show()
combined_test_loss_figures.write_image("loss_by_steps_final.pdf")

i = 0
for fig in communication_cost_upper_bound_figures:    
    for trace in fig.data:
        trace.update(line={'color': COLORS[i]}, marker=dict(symbol=MARKER_SYMBOLS[i]))
        i += 1
        combined_communication_cost_upper_bound_figures.add_trace(trace)
combined_communication_cost_upper_bound_figures.update_layout(xaxis_title="<b>Training Step (k)</b>", yaxis_title="<b>Communication Cost (GB)</b>")
combined_communication_cost_upper_bound_figures = standardize_fig(combined_communication_cost_upper_bound_figures, **fig_config)
combined_communication_cost_upper_bound_figures.update_layout(legend=dict(orientation="v", yanchor="bottom", x=0.8, y=0.2, xanchor="left", bgcolor='rgba(0,0,0,0)')) 
combined_communication_cost_upper_bound_figures.show()
combined_communication_cost_upper_bound_figures.write_image("communication_cost_upper_bound_by_steps_final.pdf")

i = 0
for fig in communication_cost_lower_bound_figures:    
    for trace in fig.data:
        trace.update(line={'color': COLORS[i]}, marker=dict(symbol=MARKER_SYMBOLS[i]))
        i += 1
        combined_communication_cost_lower_bound_figures.add_trace(trace)
combined_communication_cost_lower_bound_figures.update_layout(xaxis_title="<b>Training Step (k)</b>", yaxis_title="<b>Communication Cost (GB)</b>")
combined_communication_cost_lower_bound_figures = standardize_fig(combined_communication_cost_lower_bound_figures, **fig_config)
combined_communication_cost_lower_bound_figures.update_layout(legend=dict(orientation="v", yanchor="bottom", x=0.8, y=0.2, xanchor="left", bgcolor='rgba(0,0,0,0)')) 
combined_communication_cost_lower_bound_figures.show()
combined_communication_cost_lower_bound_figures.write_image("communication_cost_lower_bound_by_steps_final.pdf")


i = 0
for fig in test_loss_by_communication_upper_bound_cost_figures:    
    for trace in fig.data:
        trace.update(line={'color': COLORS[i]}, marker=dict(symbol=MARKER_SYMBOLS[i]))
        i += 1
        combined_test_loss_by_communication_cost_upper_bound_figures.add_trace(trace)
combined_test_loss_by_communication_cost_upper_bound_figures.update_layout(yaxis_title="<b>Validation Loss (AbsRel)</b>", xaxis_title="<b>Communication Cost (GB)</b>")
combined_test_loss_by_communication_cost_upper_bound_figures = standardize_fig(combined_test_loss_by_communication_cost_upper_bound_figures, **fig_config)
combined_test_loss_by_communication_cost_upper_bound_figures.update_layout(legend=dict(orientation="v", yanchor="bottom", x=0.8, y=0.2, xanchor="left", bgcolor='rgba(0,0,0,0)')) 
combined_test_loss_by_communication_cost_upper_bound_figures.show()
combined_test_loss_by_communication_cost_upper_bound_figures.write_image("loss_by_communication_cost_upper_bound_final.pdf")

i = 0
for fig in test_loss_by_communication_lower_bound_cost_figures:    
    for trace in fig.data:
        trace.update(line={'color': COLORS[i]}, marker=dict(symbol=MARKER_SYMBOLS[i]))
        i += 1
        combined_test_loss_by_communication_cost_lower_bound_figures.add_trace(trace)
combined_test_loss_by_communication_cost_lower_bound_figures.update_layout(yaxis_title="<b>Validation Loss (AbsRel)</b>", xaxis_title="<b>Communication Cost (GB)</b>")
combined_test_loss_by_communication_cost_lower_bound_figures = standardize_fig(combined_test_loss_by_communication_cost_lower_bound_figures, **fig_config)
combined_test_loss_by_communication_cost_lower_bound_figures.update_layout(legend=dict(orientation="v", yanchor="bottom", x=0.8, y=0.2, xanchor="left", bgcolor='rgba(0,0,0,0)')) 
combined_test_loss_by_communication_cost_lower_bound_figures.show()
combined_test_loss_by_communication_cost_lower_bound_figures.write_image("loss_by_communication_cost_lower_bound_final.pdf")